In [ ]:
import ccxt 
import pandas as pd
import ta
def fetch_data(symbol):
    binance = ccxt.binance({'enableRateLimit': True})
    data = binance.fetch_ohlcv(symbol, timeframe='1d', limit=250)
    return data
def process_data(symbol, raw_data):
    df = pd.DataFrame(raw_data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)

    # EMA & SMA
    df["ema10"] = df["close"].ewm(span=10, adjust=False).mean()
    df["sma20"] = df["close"].rolling(window=20).mean()
    df["sma50"] = df["close"].rolling(window=50).mean()
    df["sma200"] = df["close"].rolling(window=200).mean()

    # ADX, DI+, DI- dùng thư viện `ta`
    adx = ta.trend.ADXIndicator(high=df["high"], low=df["low"], close=df["close"], window=14)
    df["adx"] = adx.adx()
    df["di+"] = adx.adx_pos()
    df["di-"] = adx.adx_neg()

    df["symbol"] = symbol
    return df.reset_index()



In [119]:
import sys
import os

# Add the parent directory of 'ultils' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# timed(fetch_data("BCT/USDT"))  #



data1 =  process_data("BTC/USDT", fetch_data("BTC/USDT"))
data2 =  process_data("ETH/USDT", fetch_data("ETH/USDT"))
data3 =  process_data("NEO/USDT", fetch_data("NEO/USDT"))
data4 =  process_data("OM/USDT", fetch_data("OM/USDT"))

In [ ]:
import numpy as np
def create_embedding_sequence(df, window=30):
    #window la dod dài của chuỗi embedding
    # df_norm = df.copy()
    df_norm = df.copy().tail(window)
    for col in ['open', 'high', 'low', 'close','ema10','sma20','sma50','sma200']:
        df_norm[col] = df[col] / df['close'].iloc[-1]
    df['adx'] = df['adx'] / 100
    df['di+'] = df['di+'] / 100
    df['di-'] = df['di-'] / 100  
    df_norm["position"] = np.arange(window) / (window - 1)  
    features = [
        'open', 'high', 'low', 'close',
        'ema10', 'sma20', 'sma50', 'sma200',
        'adx', 'di+', 'di-','position'
    ]
    # remove timestamp, symbol

    df_norm = df_norm[features].tail(window)
    
    return df_norm.values.flatten()
def find_nearest_vector(query_vec, vectors):
    # query_vec: vector cần so sánh (1D)
    # vectors: mảng 2D (n_samples, n_features)
    distances = np.linalg.norm(vectors - query_vec, axis=1)
    min_index = np.argmin(distances)
    return min_index, distances[min_index]

In [97]:
data3 = create_embedding_sequence(data2,30)
data3.shape

(330,)

In [114]:
d =  data1
n =len(d)
d["position"] = np.arange(n) / (n - 1)
d.tail(10)

,timestamp,open,high,low,close,volume,ema10,sma20,sma50,sma200,adx,di+,di-,symbol,position
240,2025-07-18,119177.56,120820.71,116812.76,117924.84,19924.66266,116782.375246,112840.0730,108465.7630,97610.13790,0.002697,0.003117,0.001450,BTC/USDT,0.963855
241,2025-07-19,117924.84,118499.90,117277.34,117840.00,6635.80306,116974.670656,113314.2265,108742.8534,97731.45790,0.002765,0.003023,0.001406,BTC/USDT,0.967871
242,2025-07-20,117840.01,118856.80,116467.02,117265.12,12962.92889,117027.479628,113820.1575,108996.3182,97844.82455,0.002783,0.002844,0.001524,BTC/USDT,0.971888
243,2025-07-21,117265.11,119676.73,116515.00,117380.36,17107.33128,117091.639695,114405.1185,109231.0668,97946.80240,0.002824,0.002824,0.001405,BTC/USDT,0.975904
244,2025-07-22,117380.36,120247.80,116128.00,119954.42,20959.12973,117612.145205,114960.3595,109512.9954,98055.70360,0.002879,0.002683,0.001266,BTC/USDT,0.979920
245,2025-07-23,119954.43,120090.00,117301.00,118755.99,14558.75083,117820.116986,115418.9200,109780.5774,98158.38105,0.002929,0.002503,0.001181,BTC/USDT,0.983936
246,2025-07-24,118756.00,119450.00,117103.10,118340.99,15806.89043,117914.821170,115936.7575,110053.4600,98258.26795,0.002963,0.002359,0.001162,BTC/USDT,0.987952
247,2025-07-25,118340.98,118451.57,114723.16,117614.31,38406.34873,117860.182776,116407.5670,110375.5726,98335.16150,0.002850,0.002149,0.001628,BTC/USDT,0.991968
248,2025-07-26,117614.31,118297.35,117138.38,117919.99,6991.67206,117871.056817,116843.3745,110648.2036,98439.98840,0.002744,0.002086,0.001581,BTC/USDT,0.995984
249,2025-07-27,117919.99,118358.18,117825.50,118215.10,1189.08788,117933.610123,117340.9825,110901.4626,98555.76085,0.002649,0.002073,0.001559,BTC/USDT,1.000000


In [124]:

vec1 = create_embedding_sequence(data1, 30) # vect
vec2 = create_embedding_sequence(data2, 30)
vec3 = create_embedding_sequence(data3, 30)   
vec4 = create_embedding_sequence(data4, 30)#or cuối cùn
distances1 = np.linalg.norm(vec1 - vec2 )
distances2 = np.linalg.norm(vec1 - vec3 )
distances3 = np.linalg.norm(vec1 - vec4 )
print(f"Distance between BTC and ETH: {distances1}")
print(f"Distance between BTC and NEO: {distances2}")
print(f"Distance between BTC and OM: {distances3}")

Distance between BTC and ETH: 3.046395024650844
Distance between BTC and NEO: 2.7437703642408806
Distance between BTC and OM: 60.015254049228474


In [120]:

print (data1.tail(5))
print (data2.tail(5))

     timestamp       open       high        low      close       volume  \
245 2025-07-23  119954.43  120090.00  117301.00  118755.99  14558.75083   
246 2025-07-24  118756.00  119450.00  117103.10  118340.99  15806.89043   
247 2025-07-25  118340.98  118451.57  114723.16  117614.31  38406.34873   
248 2025-07-26  117614.31  118297.35  117138.38  117919.99   6991.67206   
249 2025-07-27  117919.99  118358.18  117825.50  118196.30   1247.55445   

             ema10        sma20        sma50       sma200        adx  \
245  117820.116986  115418.9200  109780.5774  98158.38105  29.291922   
246  117914.821170  115936.7575  110053.4600  98258.26795  29.627881   
247  117860.182776  116407.5670  110375.5726  98335.16150  28.495417   
248  117871.056817  116843.3745  110648.2036  98439.98840  27.443844   
249  117930.191941  117340.0425  110901.0866  98555.66685  26.494813   

           di+        di-    symbol  
245  25.026107  11.814949  BTC/USDT  
246  23.591866  11.621096  BTC/USDT  
24